# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd_ai 0.5.1
Uninstalling todd_ai-0.5.1:
  Successfully uninstalled todd_ai-0.5.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

## Preparation

### Models

In [3]:
@todd.registries.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.registries.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    logger=dict(),
    callbacks=[],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-06-27 09:43:31,521 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:31,524 34661:140704458489472][base.py:72 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp0ygk1y8v
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[dict(type='LogCallback', interval=5)],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-06-27 09:43:32,735 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:32,737 34661:140704458489472][base.py:72 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:32,743 34661:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:32,746 34661:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-27 09:43:32,749 34661:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-27 09:43:32,752 34661:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20])


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpwrft9i68
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:33,969 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:33,972 34661:140704458489472][base.py:72 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:33,976 34661:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([9, 1]), 'y': tensor([18,  2])} weight=0.000 loss=10.000
[2024-06-27 09:43:33,978 34661:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([7, 2]), 'y': tensor([14,  4])} weight=0.000 loss=9.000
[2024-06-27 09:43:33,979 34661:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([ 4, 10]), 'y': tensor([ 8, 20])} weight=0.000 loss=14.000
[2024-06-27 09:43:33,972 34661:140704458489472][base.py:72 todd

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:34,049 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:34,051 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:34,052 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-06-27 09:43:34,057 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([6, 3]), 'y': tensor([12,  6])} weight=0.000 loss=9.000
[2024-06-27 09:43:34,058 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5, 4]), 'y': tensor([10,  8])} weight=0.000 loss=9.000
[2024-06-27 09:43:34,051 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 ini

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:34,094 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:34,558 34661:140704458489472][log.py:49 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: adccc66
git_status: M tutorials/runners.ipynb
[2024-06-27 09:43:34,560 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:34,564 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:34,566 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-06-27 09:43:34,591 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:34,594 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:34,639 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:34,653 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-27 09:43:34,657 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-27 09:43:34,661 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpc_rzkwwm
└── log_callback
    └── 2024-06-27T09-43-34_593965-08-00.log

2 directories, 1 file

[2024-06-27 09:43:34,594 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:34,639 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:34,653 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-27 09:43:34,657 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-27 09:43:34,661 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-06-27 09:43:37,021 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:37,024 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:37,024 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:37,538 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:38,057 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-27 09:43:38,576 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': ten

In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", ema=dict(decay=0.2)),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-06-27 09:43:39,103 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:39,105 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:39,105 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:40,617 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:04 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:44,631 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:05 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-27 09:43:49,648 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': ten

In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:54,686 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:54,817 34661:140704458489472][log.py:49 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: adccc66
git_status: M tutorials/runners.ipynb
[2024-06-27 09:43:54,820 34661:140704458489472][base.py:72 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:54,825 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-27 09:43:54,830 34661:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INF

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-06-27 09:43:54,858 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:54,911 34661:140704458489472][git.py:43 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpeuttyanf/git_callback/git_diff_2024-06-27T09-43-54_911157-08-00.log
[2024-06-27 09:43:54,914 34661:140704458489472][base.py:72 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R


### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-06-27 09:43:56,238 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:56,242 34661:140704458489472][base.py:72 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,242 34661:140704458489472][base.py:72 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,252 34661:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([10,  8]), 'y': tensor([20, 16])} weight=0.000 loss=18.000
[2024-06-27 09:43:56,254 34661:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([5, 2]), 'y': tensor([10,  4])} weight=0.045 loss=6.842
[2024-06-27 09:43:56,257 34661:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter]

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:56,285 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:56,290 34661:140704458489472][base.py:72 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,351 34661:140704458489472][log.py:87 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([1, 7]), 'y': tensor([ 2, 14])} weight=0.000 loss=8.000 lr=['1.667e-03']
[2024-06-27 09:43:56,368 34661:140704458489472][log.py:87 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([10,  6]), 'y': tensor([20, 12])} weight=0.007 loss=15.947 lr=['2.333e-03']
[2024-06-27 09:43:56,371 34661:140704458489472][log.py:87 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([8, 4]), 'y': tensor([16,  8])} weight=0.025 loss=11.848 lr=['3.000e-03']
[202

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=4),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:56,404 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:56,408 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,410 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-06-27 09:43:56,408 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,410 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-06-27 09:43:56,413 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([1, 4]), 'y': tensor([2, 8])} weight=0.000 loss=5.000 lr=['1.667e-03']
[2024-06-27 09:43:

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-27 09:43:56,464 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:56,466 34661:140704458489472][lr.py:96 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-06-27 09:43:56,470 34661:140704458489472][base.py:72 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,466 34661:140704458489472][lr.py:96 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-06-27 09:43:56,470 34661:140704458489472][base.py:72 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,532 34661:140704458489472][log.py:87 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([5, 1]), 'y': tensor([10,  2])} weight=0.000 loss=6.000


### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-06-27 09:43:56,587 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:56,593 34661:140704458489472][base.py:72 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:56,597 34661:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([7, 4]), 'y': tensor([14,  8])} weight=0.000 loss=11.000
[2024-06-27 09:43:56,599 34661:140704458489472][checkpoint.py:82 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmplrmazqm3/checkpoint_callback/checkpoints/iter_1
[2024-06-27 09:43:56,605 34661:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([1, 5]), 'y': tensor([ 2, 10])} weight=0.027 loss=5.918
[2024-06-27 09:43:56,607 3466


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmplrmazqm3
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2024-06-27 09:43:58,677 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:58,678 34661:140704458489472][checkpoint.py:56 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmplrmazqm3/checkpoint_callback/checkpoints/iter_5
[2024-06-27 09:43:58,681 34661:140704458489472][base.py:69 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-27 09:43:58,682 34661:140704458489472][base.py:72 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:58,687 34661:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([4, 5]), 'y': tensor([ 8, 10])} weight=0.137 loss=8.381
[2024-06-27 09:43:58,687 34661:140704458489472][checkpoint.py:82 todd.IterBasedTraine

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-06-27 09:43:58,739 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:43:58,742 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:43:58,743 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-06-27 09:43:58,745 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([ 3, 10]), 'y': tensor([ 6, 20])} weight=0.000 loss=13.000
[2024-06-27 09:43:58,747 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([4, 8]), 'y': tensor([ 8, 16])} weight=0.032 loss=11.805
[2024-06-27 09:43:58,748 34661:140704458489472][checkpoint.py:82 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Savi


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpe14g9bk8
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-06-27 09:44:00,881 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:00,883 34661:140704458489472][checkpoint.py:56 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpe14g9bk8/checkpoint_callback/checkpoints/iter_8
[2024-06-27 09:44:00,889 34661:140704458489472][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-27 09:44:00,890 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:00,891 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-06-27 09:44:00,897 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor


--------------------



[2024-06-27 09:44:02,898 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:02,900 34661:140704458489472][checkpoint.py:56 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpe14g9bk8/checkpoint_callback/checkpoints/iter_10
[2024-06-27 09:44:02,904 34661:140704458489472][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-27 09:44:02,905 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:02,906 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-06-27 09:44:02,909 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tens

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-06-27 09:44:02,951 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:02,953 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:02,955 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-06-27 09:44:02,958 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([8, 9]), 'y': tensor([16, 18])} weight=0.000 loss=17.000
[2024-06-27 09:44:02,959 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([1, 7]), 'y': tensor([ 2, 14])} weight=0.043 loss=7.830
[2024-06-27 09:44:02,961 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter 


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpiztmcg4q
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2024-06-27 09:44:05,124 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:05,126 34661:140704458489472][checkpoint.py:56 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpiztmcg4q/checkpoint_callback/checkpoints/epoch_2
[2024-06-27 09:44:05,131 34661:140704458489472][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-27 09:44:05,132 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:05,133 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-06-27 09:44:05,126 34661:140704458489472][checkpoint.py:56 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kk

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.registries.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-06-27 09:44:05,264 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:05,268 34661:140704458489472][base.py:72 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:05,270 34661:140704458489472][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x1553e71d0>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 248, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_34661/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-06-27 09:44:05,268 34661:140704458489472][base.py:72 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:05,270 34661:140704458489472][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x1553e71d0>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 248, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_34661/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-06-27 09:44:06,737 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:06,745 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:06,747 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-06-27 09:44:06,752 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([ 6, 10]), 'y': tensor([12, 20])} weight=0.000 loss=16.000
[2024-06-27 09:44:06,756 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([7, 1]), 'y': tensor([14,  2])} weight=0.040 loss=7.840
[2024-06-27 09:44:06,758 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from 


--------------------



[2024-06-27 09:44:08,837 34661:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-27 09:44:08,839 34661:140704458489472][base.py:72 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-27 09:44:08,840 34661:140704458489472][base.py:84 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpfap2tj6v/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-06-27 09:44:08,842 34661:140704458489472][base.py:69 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-27 09:44:08,843 34661:140704458489472][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-06-27 09:44:08,846 34661:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from 

## Dry Run

In [28]:
todd.Store.DRY_RUN = True